# Use Data Loader to Get Sample, Apply Selection and Make RSE List 

##### Set the run and sample you wish to look at

In [1]:
Run = 1
SampleType = "bnb" # Choose from bnb, ext, nu, nue, drt

##### Import the data loader

In [2]:
# Import the data loader
%load_ext autoreload
%autoreload 2
import data_loading as dl
from importlib import reload
reload(dl)
import os

##### Load the sample with the new data loader

In [3]:
reload(dl)
df_bnb = dl.load_sample(Run, "runs", SampleType, loadpi0variables=True, loadshowervariables=True, loadrecoveryvars=True,loadsystematics=True)

/uboone/app/users/cthorpe/RefactoredPELEE/data_loading.py:1015: RuntimeWarning: overflow encountered in multiply
  mc_Q2 = -1 * (mc_q_E * mc_q_E - mc_q_px * mc_q_px - mc_q_py * mc_q_py - mc_q_pz * mc_q_pz)
/uboone/app/users/cthorpe/RefactoredPELEE/data_loading.py:1015: RuntimeWarning: invalid value encountered in subtract
  mc_Q2 = -1 * (mc_q_E * mc_q_E - mc_q_px * mc_q_px - mc_q_py * mc_q_py - mc_q_pz * mc_q_pz)
/uboone/app/users/cthorpe/RefactoredPELEE/data_loading.py:534: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(2 * e1 * e2 * (1 - px1 * px2 - py1 * py2 - pz1 * pz2))
/uboone/app/users/cthorpe/RefactoredPELEE/data_loading.py:534: RuntimeWarning: overflow encountered in float_scalars
  return np.sqrt(2 * e1 * e2 * (1 - px1 * px2 - py1 * py2 - pz1 * pz2))
/uboone/app/users/cthorpe/Mambaforge/envs/python3LEE/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs

##### Configure the path to the file we're looking at

In [6]:
path = dl.get_path(run_number=Run,category="runs",dataset=SampleType)
filename = dl.get_filename(run_number=Run,category="runs",dataset=SampleType)
print("path=",path)
print("filename=",filename)

path= /uboone/data/users/cthorpe/PELEE/run1/nuepresel/
filename= run1_nuepresel.root


##### Import the selection algorithms and plotter objects

In [7]:
from unblinding_far_sideband import *
import localSettings as ls

##### Prepare the selection query

In [8]:
# choose sideband, preselection, and selection
# find defintions in unblinding_far_sideband.py
sdb_key = 'None' #'None'
pre_key = 'ZP'
sel_key = 'ZPBDT'
#sdb_key = 'TwoPShr'
#pre_key = 'NP'
#sel_key = 'NPBDTAllShr' #'None' 'NPVLAllShr' 'NPLAllShr' 'NPBDTAllShr' 'NPTAllShr'
run = 1

sideband = sideband_categories[sdb_key]
preselection = preselection_categories[pre_key]
sel =  selection_categories[sel_key]

DETSYSPATH = ls.ntuple_path+'/detsys/{}_{}_{}/'.format(sideband['dir'], preselection['dir'], sel['dir'])
DETSYS = {'mc':False,'nue':False,'ccpi0':False,'ncpi0':False,'dirt':False,\
          'ncnopi':False,'nccpi':False,'ncpi0':False,'ccpi0':False}


QUERY = ""
for query in [sideband['query'],preselection['query'],sel['query']]:
    if query is None: continue
    if QUERY: QUERY += ' and '
    QUERY += query
    if run in [1, 2, 3, 12]:
        QUERY += f' and run{run}==True'

#print("Selection Used:",sdb_key+"_"+pre_key+"_"+sel_key)

##### Write the rse list of selected events to the file

In [9]:
# Format the name of the output file
selection=sdb_key + "_" + pre_key + "_" + sel_key

# Make a subdirectory of the one containing the samples
os.system("mkdir -p " + path + "/" + selection)

# Format the name of the new file
new_filename = path + "/" + selection + "/rse_" + filename + ".txt"
print("Output file is:",new_filename)

# Write the rse list to the output file
df_bnb_sel = df_bnb.query(QUERY)
outfile = open(new_filename,'a')
outfile.write(df_bnb_sel[['run','sub','evt']].to_string(index=False,header=False))
outfile.close()

Output file is: /uboone/data/users/cthorpe/PELEE/run1/nuepresel//None_ZP_ZPBDT/rse_run1_nuepresel.root.txt
